In [1]:
import os
import rootutils

rootutils.setup_root(os.getcwd(), indicator=".project-root", pythonpath=True)

from torchvision import transforms
import torch
from src.data.components.graphs_datamodules import (
    IMCBaseDictTransform,
    PickleDataset,
    PatchAugmentations,
)
from src.data.imc_datamodule import add_channel
import src.data.components.graphs_datamodules as gd
from torchvision.datasets import MNIST
from torch.utils.data import ConcatDataset, Dataset, random_split

In [2]:
from pathlib import Path

In [7]:
from src.models.components.plot import restore_tensor
import matplotlib.pyplot as plt

base_transforms = IMCBaseDictTransform()

aug_transforms_train = gd.PatchAugmentations(
    prob=1.0,
    size=13,
    patch_size=1,
)

aug_transforms_val = gd.PatchAugmentations(
    prob=1.0,
    size=13,
    patch_size=1,
    is_validation=True,
)

dual_transforms_train = gd.DualOutputTransform(base_transforms, aug_transforms_train)

dual_transforms_val = gd.DualOutputTransform(base_transforms, aug_transforms_val)

train_path = Path("../data") / 'IMC-sample' / 'train.h5'
test_path = Path("../data") / 'IMC-sample' / 'test.h5'
trainset = PickleDataset(train_path, transform=dual_transforms_train)
testset = PickleDataset(train_path, transform=dual_transforms_val)
train_ratio, val_ratio, test_ratio, leftover_ratio = [3600, 1044, 0, 0]
size_testset = len(testset)
size_trainset = len(trainset)
data_train, _ = random_split(
    dataset=trainset,
    lengths=[train_ratio, size_trainset - train_ratio],
    generator=torch.Generator().manual_seed(42),
)
# dataset = ConcatDataset(datasets=[trainset, testset])
data_val, data_test, _ = random_split(
    dataset=testset,
    lengths=[val_ratio, test_ratio, size_testset - val_ratio - test_ratio],
    generator=torch.Generator().manual_seed(42),
)

train_dataset = gd.GridGraphDataset(grid_size=13, dataset=data_train, channels=list(range(10)))

train_loader = gd.DenseGraphDataLoader(
    dataset=train_dataset,
    batch_size=8,
    num_workers=7,
    pin_memory=False,
    persistent_workers=7 > 0,
)

In [8]:
for el in train_loader:
    print(el)
    break

In [9]:
el.node_features.shape

torch.Size([64, 169, 10])

In [ ]:
el.

tensor([[  0,   1,   2,  ..., 166, 167, 168],
        [  0,   1,   2,  ..., 166, 167, 168],
        [  0,   1,   2,  ..., 166, 167, 168],
        ...,
        [  0,   1,   2,  ..., 166, 167, 168],
        [  0,   1,   2,  ..., 166, 167, 168],
        [  0,   1,   2,  ..., 166, 167, 168]])